In [92]:
# Breast classification 
import pandas as pd
import torch
from VQCStronglyEntanglingModel import VQCStronglyEntanglingModel

# Load data and preprocessing

from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler

import torch
import pandas as pd
from torch import nn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import requests
import telegram_send
import sys

In [93]:
numElements = 1000
numFeatures = 2
encoding = "XY"
numLayers = 4
reup = True

X_train = pd.read_csv(f"../ReducedDataset/Train_{numElements}_{numFeatures}.csv", header=None, index_col=False)
X_test = pd.read_csv(f"../ReducedDataset/Test_{numElements}_{numFeatures}.csv", header=None, index_col=False)
y_train = X_train.pop(2)
y_test = X_test.pop(2)
y_train = torch.from_numpy(np.array(y_train)).type(torch.float)
y_test = torch.from_numpy(np.array(y_test)).type(torch.float)

minmaxScaler = MinMaxScaler(feature_range=(-1, 1))

X_train = minmaxScaler.fit_transform(X_train)
X_test = minmaxScaler.transform(X_test)


numWires = X_train.shape[1]

X_train = torch.from_numpy(X_train).type(torch.float)
X_test = torch.from_numpy(X_test).type(torch.float)
train_tensor = torch.utils.data.TensorDataset(X_train, y_train)
test_tensor = torch.utils.data.TensorDataset(X_test, y_test)


BATCH_SIZE = 2

train_dataloader = DataLoader(train_tensor, 
                              batch_size=BATCH_SIZE,
                              shuffle=True)


test_dataloader = DataLoader(test_tensor, 
                              batch_size=BATCH_SIZE,
                              shuffle=False)

In [94]:
MODEL_SAVE_PATH = f"Strongly_{numElements}_{numFeatures}_{encoding}_{numLayers}_{reup}.pth"

In [95]:
# define the model class
class FraudDetection(nn.Module):
    # parameters which can be set:
    # numLayerStrongly (int) : set the number of layer for stronglyEntangling ansatz for the VQC model
    # encoding (str): set the type of encoding to use (accepted "angle" or "amplitude")
    # gates (str): set the type of encoding gates used for the VQC  
    def __init__(self,
                numWires : int,
                numLayerStrongly: int,
                encoding: str,
                gates: str,
                reuploading: bool):
        super().__init__()
        # for the VQC, the number of classes is fixed to 2 considering the binary classification
        # the number of wires is equal to 5 resulting input vector is composed of 5 elements
        # reuploading in this case is not applied
        self.layer1 = VQCStronglyEntanglingModel(
            encoding=encoding,
            numLayerStrongly=numLayerStrongly,
            numClasses=2,
            numWires=numWires,
            gates=gates,
            reUploading=reuploading
        )
    # the input vector is firstly passed to the Kernel module and then to the VQC
    def forward(self,x):
        return self.layer1(x)

In [96]:
model_0 = FraudDetection(
    numWires=numWires,
    numLayerStrongly = numLayers,
    encoding = "angle",
    gates=encoding,
    reuploading=reup
)

In [97]:
model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [98]:
def conf_matrix(y_true, y_pred):
    res = np.zeros((2, 2))
    for pred, true in zip(y_pred, y_true):
        if true == 0 and pred == 0:
            res[0][0] += 1
        elif true == 1 and pred == 1:
            res[1][1] += 1
        elif true == 0 and pred == 1:
            res[0][1] += 1
        else:
            res[1][0] += 1
    return res

In [122]:

confMatrixTest = np.zeros((2,2))
with torch.inference_mode():
    for X_train, y_train in train_dataloader:
        test_logits = model_0(X_test).squeeze()
        test_pred = torch.sigmoid(test_logits)
        test_pred = np.array([1 if p >= 0.015 else 0 for p in test_pred])
        confMatrixTest += conf_matrix(y_pred=test_pred, y_true=y_train.numpy())
confMatrixTest

array([[  0., 881.],
       [  0.,  19.]])